### Day 14: Restroom Redoubt

Link: https://adventofcode.com/2024/day/14#part2

To solve part two, we first need an insight from the input: all robots return to their original position after `grid height * grid width seconds`. This means we have at most the grid area number of possible states to cover. Given the input size of 101 x 103, this results in only 10,403 states to analyze and locate the earliest one resembling a Christmas tree. If we take one second per state, it should take approximately three hours.

Or, we can sort the states using a function that scores each state's randomness, which you can find on the internet. States resembling images are likely to score as less random, bringing the Christmas tree state closer to the top, allowing us to find it sooner.

The code below generates all possible states as files in the outputs folder, sorted by randomness. In my case, the Christmas tree state was the first file. We can determine the second at which that state occurs from the filename.

In [ ]:
!pip install numpy==2.2.0

In [ ]:
# Please ensure there is an `input.txt` file in this folder containing your input.
with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
import os
import shutil
import re
import numpy as np
from dataclasses import dataclass


@dataclass
class Position:
    x: int
    y: int


@dataclass
class Robot:
    position: Position
    velocity: Position


grid_width = 101
grid_height = 103
robots: list[Robot] = []


def get_position(text: str, pattern: str) -> Position:
    match = re.search(pattern, text)
    x = int(match.group(1))
    y = int(match.group(2))
    return Position(x=x, y=y)


for line in lines:
    position = get_position(line, r"p=(\d+),(\d+)")
    velocity = get_position(line, r"v=(-?\d+),(-?\d+)")
    robot = Robot(position=position, velocity=velocity)
    robots.append(robot)


OUTPUTS_FOLDER = "outputs"


# Clean outputs folder
if os.path.exists(OUTPUTS_FOLDER):
    shutil.rmtree(OUTPUTS_FOLDER)


os.makedirs(OUTPUTS_FOLDER)


@dataclass
class State:
    second: int
    grid: np.ndarray


def make_state(second: int) -> State:
    grid = np.zeros((grid_height, grid_width), dtype=int)

    for robot in robots:
        grid[robot.position.y][robot.position.x] = 1

    return State(second=second, grid=grid)


# Generate all possible
max_iterations = grid_height * grid_width
states: list[State] = []


for second in range(1, max_iterations + 1):
    for robot in robots:
        robot.position.x = (robot.position.x + robot.velocity.x) % grid_width
        robot.position.y = (robot.position.y + robot.velocity.y) % grid_height

    state = make_state(second)
    states.append(state)


def randomness_score(state: State) -> tuple[float, float]:
    density = state.grid.sum() / state.grid.size
    filled_positions = np.argwhere(state.grid)
    spread = np.var(filled_positions, axis=0).sum() if filled_positions.size > 0 else 0
    return density, spread


def sort_states_by_randomness(states: list[State]) -> list[State]:
    return sorted(states, key=lambda state: randomness_score(state), reverse=True)


states = sort_states_by_randomness(states)


for idx, state in enumerate(states):
    length_digits = len(str(max_iterations))
    file_path = f"{OUTPUTS_FOLDER}/{str(idx).zfill(length_digits)}-second-{str(state.second).zfill(length_digits)}.txt"
    output_grid = [[" "] * grid_width for _ in range(grid_height)]

    for row in range(grid_height):
        for column in range(grid_width):
            if state.grid[row][column]:
                output_grid[row][column] = "*"

    output_lines = ["".join(output_row) + "\n" for output_row in output_grid]

    with open(file_path, "w") as file:
        file.writelines(output_lines)